<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>

You might want to consider the [start](search.ipynb) of this tutorial.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [3]:
VERSION = '2017'

In [4]:
A = use('bhsa', hoist=globals(), version=VERSION)

	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in /Users/dirk/text-fabric-data/annotation/app-bhsa/code:
	#d3cf8f0c2ab5d690a0fda14ea31c33da5c5c8483 (latest commit)
	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/bhsa/tf/2017:
	rv1.6 (latest release)
	connecting to online GitHub repo etcbc/phono ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/phono/tf/2017:
	r1.2=#1ac68e976ee4a7f23eb6bb4c6f401a033d0ec169 (latest release)
	connecting to online GitHub repo etcbc/parallels ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/parallels/tf/2017:
	r1.2=#395dfe2cb69c261862fab9f0289e594a52121d5c (latest release)


# Relationships

There are relationship between objects that are about their identities, the way they 
occupy space, and the way they are connected.

Are two objects the same, do they occupy the same slots, do they overlap, is one embedded in the other,
does one come before the other? Is there an edge from one to another?

Although these relationships are easy to define, and even easy to implement,
they may be very costly to use. 
When searching, most of them have to be computed very many times.

Some of them have been precomputed and stored in an index, e.g. the embedding relationships.
They can be used without penalty.

Other relations are not suitable for pre-computing: most inequality relations are of that kind.
It would require an enormous amount of storage to pre-compute for each node the set of nodes that
occupy different slots. This type of relation will not be used in narrowing down the search space,
which means that it may take more time to get the results.

We are going to test all of our relationships here.

Let us first see what relationships we have:

In [5]:
S.relationsLegend()

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

The top of the list are identity and spatial relationships.
We are going to dicuss them. They are documented in
[relationships](https://annotation.github.io/text-fabric/Use/Search/#relational-operators)

The bottom of the list are relationships defined by the edge features of your dataset.
We have discussed them in [advanced](searchAdvanced.ipynb).

# Identity and order

## = equal as node

The `=` means that both parts are the same node. Left and right are not two things with similar properties,
no, they are one and the same thing.

Useful if the thing you search for it part of two wildly different patterns.

In [6]:
query = '''
v1:verse
  sentence
    clause rela=Objc
      phrase
        word sp=verb gn=f nu=pl
v2:verse
  sentence
    c1:clause
    c2:clause
    c3:clause
    c1 < c2
    c2 < c3
v1 = v2
'''
results = A.search(query)
A.table(results, end=3)

  1.17s 48 results


n,p,verse,sentence,clause,phrase,word,verse,sentence,clause,clause,clause
1,Jeremiah 44:15,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים,מְקַטְּרֹ֤ות,מְקַטְּרֹ֤ות,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם,הַיֹּֽדְעִים֙,כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים
2,Jeremiah 44:15,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים,מְקַטְּרֹ֤ות,מְקַטְּרֹ֤ות,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם,הַיֹּֽדְעִים֙,הָעֹמְדֹ֖ות
3,Jeremiah 44:15,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים,מְקַטְּרֹ֤ות,מְקַטְּרֹ֤ות,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים הַיֹּֽדְעִים֙ כִּֽי־מְקַטְּרֹ֤ות נְשֵׁיהֶם֙ לֵאלֹהִ֣ים אֲחֵרִ֔ים וְכָל־הַנָּשִׁ֥ים הָעֹמְדֹ֖ות קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס לֵאמֹֽר׃,וַיַּעֲנ֣וּ אֶֽת־יִרְמְיָ֗הוּ כָּל־הָאֲנָשִׁ֤ים וְכָל־הַנָּשִׁ֥ים קָהָ֣ל גָּדֹ֑ול וְכָל־הָעָ֛ם,הַיֹּֽדְעִים֙,הַיֹּשְׁבִ֥ים בְּאֶֽרֶץ־מִצְרַ֖יִם בְּפַתְרֹ֥וס


In [7]:
A.show(results, start=1, end=1, colorMap={3: 'magenta', 7: 'cyan'})

## # unequal as node

`n # m` if `n` and `m` are not the same node.

If you write a template, and you know that one node should come before another one,
consider using `<` or `>`, which will constrain the results better.

We have seen this in action in the search for gapped phrases.

## < and > canonical order

`n < m` if `n` comes before `m` in the
[canonical ordering](https://annotation.github.io/text-fabric/Api/General/#navigating-nodes)
of nodes.

We have seen them in action before.

# Space occupation

We show that the following relationships also work with custom sets, as introduced in the 
[searchAdvanced tutorial](searchAdvanced.ipynb).

We make two custom sets, `common` and `rare`, consisting of nodes whose contained slots have all
common lexemes or some rare lexemes, like we did in
[searchAdvanced](searchAdvanced.ipynb).

In [8]:
COMMON_RANK = 100
RARE_RANK = 500

common = set()
rare = set()

for n in N():
    nTp = F.otype.v(n)
    if nTp == 'lex':
       continue
    if nTp == 'word':
        ranks = [F.rank_lex.v(n)]
    else:
        ranks = [F.rank_lex.v(w) for w in L.d(n, otype='word')]
    maxRank = max(ranks)
    minRank = min(ranks)
    if maxRank < COMMON_RANK:
        common.add(n)
    if maxRank > RARE_RANK:
        rare.add(n)
        
print(f'{len(common):>6} members in set common')
print(f'{len(rare):>6} members in set rare')

669178 members in set common
424746 members in set rare


Now we can do all kinds of searches within the domain of `common` and `rare` things.

We give the names to all the sets and put them in a dictionary.

In [9]:
customSets=dict(
    common=common,
    rare=rare,
)

**Expert remark**

Note that these sets contain both slot nodes and non-slot nodes. 
The code that implements the basic relationship is heavily optimized and contains case distinctions as to whether nodes are slot
or non-slot.
For ordinary node types, it is clear on beforehand whether its nodes are slot or non-slot, but custom sets may contain both.

So our `frequent` and `infrequent` sets are good tests whether the basic relationships are correctly implemented.

## == same slots

Two objects are extensionally equal if they occupy exactly the same slots.

In [10]:
indent(reset=True)
query = '''
v:verse
    s:sentence
v == s
'''
results = A.search(query)

  0.93s 3601 results


In [11]:
A.table(results, end=10)
A.show(results, start=1, end=1)
info('Done')

n,p,verse,sentence
1,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃
2,Genesis 1:16,וַיַּ֣עַשׂ אֱלֹהִ֔ים אֶת־שְׁנֵ֥י הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים אֶת־הַמָּאֹ֤ור הַגָּדֹל֙ לְמֶמְשֶׁ֣לֶת הַיֹּ֔ום וְאֶת־הַמָּאֹ֤ור הַקָּטֹן֙ לְמֶמְשֶׁ֣לֶת הַלַּ֔יְלָה וְאֵ֖ת הַכֹּוכָבִֽים׃,וַיַּ֣עַשׂ אֱלֹהִ֔ים אֶת־שְׁנֵ֥י הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים אֶת־הַמָּאֹ֤ור הַגָּדֹל֙ לְמֶמְשֶׁ֣לֶת הַיֹּ֔ום וְאֶת־הַמָּאֹ֤ור הַקָּטֹן֙ לְמֶמְשֶׁ֣לֶת הַלַּ֔יְלָה וְאֵ֖ת הַכֹּוכָבִֽים׃
3,Genesis 2:1,וַיְכֻלּ֛וּ הַשָּׁמַ֥יִם וְהָאָ֖רֶץ וְכָל־צְבָאָֽם׃,וַיְכֻלּ֛וּ הַשָּׁמַ֥יִם וְהָאָ֖רֶץ וְכָל־צְבָאָֽם׃
4,Genesis 2:9,וַיַּצְמַ֞ח יְהוָ֤ה אֱלֹהִים֙ מִן־הָ֣אֲדָמָ֔ה כָּל־עֵ֛ץ נֶחְמָ֥ד לְמַרְאֶ֖ה וְטֹ֣וב לְמַאֲכָ֑ל וְעֵ֤ץ הַֽחַיִּים֙ בְּתֹ֣וךְ הַגָּ֔ן וְעֵ֕ץ הַדַּ֖עַת טֹ֥וב וָרָֽע׃,וַיַּצְמַ֞ח יְהוָ֤ה אֱלֹהִים֙ מִן־הָ֣אֲדָמָ֔ה כָּל־עֵ֛ץ נֶחְמָ֥ד לְמַרְאֶ֖ה וְטֹ֣וב לְמַאֲכָ֑ל וְעֵ֤ץ הַֽחַיִּים֙ בְּתֹ֣וךְ הַגָּ֔ן וְעֵ֕ץ הַדַּ֖עַת טֹ֥וב וָרָֽע׃
5,Genesis 3:5,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃
6,Genesis 3:23,וַֽיְשַׁלְּחֵ֛הוּ יְהוָ֥ה אֱלֹהִ֖ים מִגַּן־עֵ֑דֶן לַֽעֲבֹד֙ אֶת־הָ֣אֲדָמָ֔ה אֲשֶׁ֥ר לֻקַּ֖ח מִשָּֽׁם׃,וַֽיְשַׁלְּחֵ֛הוּ יְהוָ֥ה אֱלֹהִ֖ים מִגַּן־עֵ֑דֶן לַֽעֲבֹד֙ אֶת־הָ֣אֲדָמָ֔ה אֲשֶׁ֥ר לֻקַּ֖ח מִשָּֽׁם׃
7,Genesis 4:24,כִּ֥י שִׁבְעָתַ֖יִם יֻקַּם־קָ֑יִן וְלֶ֖מֶךְ שִׁבְעִ֥ים וְשִׁבְעָֽה׃,כִּ֥י שִׁבְעָתַ֖יִם יֻקַּם־קָ֑יִן וְלֶ֖מֶךְ שִׁבְעִ֥ים וְשִׁבְעָֽה׃
8,Genesis 5:23,וַיְהִ֖י כָּל־יְמֵ֣י חֲנֹ֑וךְ חָמֵ֤שׁ וְשִׁשִּׁים֙ שָׁנָ֔ה וּשְׁלֹ֥שׁ מֵאֹ֖ות שָׁנָֽה׃,וַיְהִ֖י כָּל־יְמֵ֣י חֲנֹ֑וךְ חָמֵ֤שׁ וְשִׁשִּׁים֙ שָׁנָ֔ה וּשְׁלֹ֥שׁ מֵאֹ֖ות שָׁנָֽה׃
9,Genesis 6:5,וַיַּ֣רְא יְהוָ֔ה כִּ֥י רַבָּ֛ה רָעַ֥ת הָאָדָ֖ם בָּאָ֑רֶץ וְכָל־יֵ֨צֶר֙ מַחְשְׁבֹ֣ת לִבֹּ֔ו רַ֥ק רַ֖ע כָּל־הַיֹּֽום׃,וַיַּ֣רְא יְהוָ֔ה כִּ֥י רַבָּ֛ה רָעַ֥ת הָאָדָ֖ם בָּאָ֑רֶץ וְכָל־יֵ֨צֶר֙ מַחְשְׁבֹ֣ת לִבֹּ֔ו רַ֥ק רַ֖ע כָּל־הַיֹּֽום׃
10,Genesis 6:8,וְנֹ֕חַ מָ֥צָא חֵ֖ן בְּעֵינֵ֥י יְהוָֽה׃ פ,וְנֹ֕חַ מָ֥צָא חֵ֖ן בְּעֵינֵ֥י יְהוָֽה׃ פ


  5.35s Done


Now a similar query with the custom sets:

In [12]:
indent(reset=True)
query = '''
v:common otype=verse
    s:common otype=sentence
v == s
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  1.52s 135 results


n,p,verse,sentence
1,Exodus 6:10,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
2,Exodus 6:28,וַיְהִ֗י בְּיֹ֨ום דִּבֶּ֧ר יְהוָ֛ה אֶל־מֹשֶׁ֖ה בְּאֶ֥רֶץ מִצְרָֽיִם׃ פ,וַיְהִ֗י בְּיֹ֨ום דִּבֶּ֧ר יְהוָ֛ה אֶל־מֹשֶׁ֖ה בְּאֶ֥רֶץ מִצְרָֽיִם׃ פ
3,Exodus 13:1,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
4,Exodus 14:1,וַיְדַבֵּ֥ר יְהֹוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהֹוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
5,Exodus 16:11,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
6,Exodus 20:1,וַיְדַבֵּ֣ר אֱלֹהִ֔ים אֵ֛ת כָּל־הַדְּבָרִ֥ים הָאֵ֖לֶּה לֵאמֹֽר׃ ס,וַיְדַבֵּ֣ר אֱלֹהִ֔ים אֵ֛ת כָּל־הַדְּבָרִ֥ים הָאֵ֖לֶּה לֵאמֹֽר׃ ס
7,Exodus 25:1,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
8,Exodus 30:11,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
9,Exodus 30:17,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃
10,Exodus 30:22,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃,וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה לֵּאמֹֽר׃


  1.53s Done


As a check we compute manually the maximum rank of the lexemes in the clauses yielded by the query without the custom sets:

In [13]:
rejected = 0
for (verse, sentence) in results:
    maxRank = max(F.rank_lex.v(w) for w in L.d(verse, otype='word'))
    if maxRank >= COMMON_RANK:
        rejected += 1
        
print(f'Rejected {rejected} non-common results, leaving {len(results) - rejected} ones')

Rejected 3466 non-common results, leaving 135 ones


## && overlap

Two objects overlap if and only if they share at least one slot.
This is quite costly to use in some cases.

We are going to find the sentences that overlap with two verses.

In [14]:
query = '''
sentence
/with/
v1:verse
&& ..
v2:verse
&& ..
v1 < v2
/-/
'''

Explanation: the query looks for sentences and delivers results that are tuples with only a sentence node.

This is because the stuff within the `/with/` quantifier does not contribute to the result tuples.

The `/with/` quantifier poses a few restrictions on its parent, the `sentence`.

From within the quantifier you can refer to the parent by `..`.

The condition is that there are verses `v1` and `v2` that have overlap with the sentence, and that `v1` comes before `v2`.

The result is a tuple of exactly the sentences that span multiple verses.

In [15]:
results = A.search(query)

  2.55s 892 results


In [16]:
A.table(results, end=5)
A.show(results, condensed=False, start=1, end=3)

n,p,sentence
1,Genesis 1:17,וַיִּתֵּ֥ן אֹתָ֛ם אֱלֹהִ֖ים בִּרְקִ֣יעַ הַשָּׁמָ֑יִם לְהָאִ֖יר עַל־הָאָֽרֶץ׃ וְלִמְשֹׁל֙ בַּיֹּ֣ום וּבַלַּ֔יְלָה וּֽלֲהַבְדִּ֔יל בֵּ֥ין הָאֹ֖ור וּבֵ֣ין הַחֹ֑שֶׁךְ
2,Genesis 1:29,הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֲשֶׁר־בֹּו֙ נֶ֣פֶשׁ חַיָּ֔ה אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה
3,Genesis 2:4,בְּיֹ֗ום עֲשֹׂ֛ות יְהוָ֥ה אֱלֹהִ֖ים אֶ֥רֶץ וְשָׁמָֽיִם׃ וַיִּיצֶר֩ יְהוָ֨ה אֱלֹהִ֜ים אֶת־הָֽאָדָ֗ם עָפָר֙ מִן־הָ֣אֲדָמָ֔ה
4,Genesis 7:2,מִכֹּ֣ל׀ הַבְּהֵמָ֣ה הַטְּהֹורָ֗ה תִּֽקַּח־לְךָ֛ שִׁבְעָ֥ה שִׁבְעָ֖ה אִ֣ישׁ וְאִשְׁתֹּ֑ו וּמִן־הַבְּהֵמָ֡ה אֲ֠שֶׁר לֹ֣א טְהֹרָ֥ה הִ֛וא שְׁנַ֖יִם אִ֥ישׁ וְאִשְׁתֹּֽו׃ גַּ֣ם מֵעֹ֧וף הַשָּׁמַ֛יִם שִׁבְעָ֥ה שִׁבְעָ֖ה זָכָ֣ר וּנְקֵבָ֑ה לְחַיֹּ֥ות זֶ֖רַע עַל־פְּנֵ֥י כָל־הָאָֽרֶץ׃
5,Genesis 7:8,מִן־הַבְּהֵמָה֙ הַטְּהֹורָ֔ה וּמִן־הַ֨בְּהֵמָ֔ה אֲשֶׁ֥ר אֵינֶ֖נָּה טְהֹרָ֑ה וּמִ֨ן־הָעֹ֔וף וְכֹ֥ל אֲשֶׁר־רֹמֵ֖שׂ עַל־הָֽאֲדָמָֽה׃ שְׁנַ֨יִם שְׁנַ֜יִם בָּ֧אוּ אֶל־נֹ֛חַ אֶל־הַתֵּבָ֖ה זָכָ֣ר וּנְקֵבָ֑ה כַּֽאֲשֶׁ֛ר צִוָּ֥ה אֱלֹהִ֖ים אֶת־נֹֽחַ׃


Now with custom sets:

In [17]:
query = '''
common otype=sentence
/with/
v1:verse
&& ..
v2:verse
&& ..
v1 < v2
/-/
'''

In [18]:
resultsCommon = A.search(query, sets=customSets)

  1.35s 6 results


In [19]:
A.table(resultsCommon)

n,p,sentence
1,Numbers 27:8,אִ֣ישׁ כִּֽי־יָמ֗וּת וּבֵן֙ אֵ֣ין לֹ֔ו וְאִם־אֵ֥ין לֹ֖ו בַּ֑ת וְאִם־אֵ֥ין לֹ֖ו אַחִ֑ים וְאִם־אֵ֣ין אַחִים֮ לְאָבִיו֒
2,Jeremiah 7:24,וְלֹ֤א שָֽׁמְעוּ֙ לְמִן־הַיֹּ֗ום אֲשֶׁ֨ר יָצְא֤וּ אֲבֹֽותֵיכֶם֙ מֵאֶ֣רֶץ מִצְרַ֔יִם עַ֖ד הַיֹּ֣ום הַזֶּ֑ה
3,Jeremiah 21:8,וְאֶל־הָעָ֤ם הַזֶּה֙ תֹּאמַ֔ר וּלְבֵית֙ מֶ֣לֶךְ יְהוּדָ֔ה
4,Jeremiah 21:11,שִׁמְע֖וּ דְּבַר־יְהוָֽה׃ בֵּ֣ית דָּוִ֗ד
5,Ezekiel 37:18,וְכַֽאֲשֶׁר֙ יֹאמְר֣וּ אֵלֶ֔יךָ בְּנֵ֥י עַמְּךָ֖ לֵאמֹ֑ר דַּבֵּ֣ר אֲלֵהֶ֗ם
6,Psalms 25:1,אֵלֶ֥יךָ יְ֝הוָ֗ה נַפְשִׁ֥י אֶשָּֽׂא׃ אֱֽלֹהַ֗י


The following query is only a tad faster, but less insightful.

In [20]:
query = '''
sentence
  =: w1:word
  w2:word
  :=

v1:verse
  wv1:word

v2:verse
  wv2:word

w1 = wv1
w2 = wv2
v1 < v2
'''

In [21]:
indent(reset=True)
fastResults = A.search(query)
info('Done')

  2.32s 892 results
  2.32s Done


In [22]:
sorted((x[0],) for x in fastResults) == sorted(results)

True

## ## not the same slots

True when the two objects in question do not occupy exactly the same set of slots.
This is a very loose relationship.

We look for sentences that start with a sentence atom that is not co-extensive with its sentence.

In [23]:
indent(reset=True)
query = '''
verse
  s:sentence
    =: sa:sentence_atom
  s ## sa
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, start=1, end=1)
info('Done')

  0.55s 454 results


n,p,verse,sentence,sentence_atom
1,Genesis 3:3,וּמִפְּרִ֣י הָעֵץ֮ אֲשֶׁ֣ר בְּתֹוךְ־הַגָּן֒ אָמַ֣ר אֱלֹהִ֗ים לֹ֤א תֹֽאכְלוּ֙ מִמֶּ֔נּוּ וְלֹ֥א תִגְּע֖וּ בֹּ֑ו פֶּן־תְּמֻתֽוּן׃,וּמִפְּרִ֣י הָעֵץ֮ אֲשֶׁ֣ר בְּתֹוךְ־הַגָּן֒ לֹ֤א תֹֽאכְלוּ֙ מִמֶּ֔נּוּ,וּמִפְּרִ֣י הָעֵץ֮ אֲשֶׁ֣ר בְּתֹוךְ־הַגָּן֒
2,Genesis 13:1,וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־אֲשֶׁר־לֹ֛ו וְלֹ֥וט עִמֹּ֖ו הַנֶּֽגְבָּה׃,וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־אֲשֶׁר־לֹ֛ו הַנֶּֽגְבָּה׃,וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ה֠וּא וְאִשְׁתֹּ֧ו וְכָל־אֲשֶׁר־לֹ֛ו
3,Genesis 13:9,הֲלֹ֤א כָל־הָאָ֨רֶץ֙ לְפָנֶ֔יךָ הִפָּ֥רֶד נָ֖א מֵעָלָ֑י אִם־הַשְּׂמֹ֣אל וְאֵימִ֔נָה וְאִם־הַיָּמִ֖ין וְאַשְׂמְאִֽילָה׃,הִפָּ֥רֶד נָ֖א מֵעָלָ֑י אִם־הַשְּׂמֹ֣אל וְאִם־הַיָּמִ֖ין,הִפָּ֥רֶד נָ֖א מֵעָלָ֑י אִם־הַשְּׂמֹ֣אל
4,Genesis 21:14,וַיַּשְׁכֵּ֣ם אַבְרָהָ֣ם׀ בַּבֹּ֡קֶר וַיִּֽקַּֽח־לֶחֶם֩ וְחֵ֨מַת מַ֜יִם וַיִּתֵּ֣ן אֶל־הָ֠גָר שָׂ֧ם עַל־שִׁכְמָ֛הּ וְאֶת־הַיֶּ֖לֶד וַֽיְשַׁלְּחֶ֑הָ וַתֵּ֣לֶךְ וַתֵּ֔תַע בְּמִדְבַּ֖ר בְּאֵ֥ר שָֽׁבַע׃,וַיִּֽקַּֽח־לֶחֶם֩ וְחֵ֨מַת מַ֜יִם וְאֶת־הַיֶּ֖לֶד,וַיִּֽקַּֽח־לֶחֶם֩ וְחֵ֨מַת מַ֜יִם
5,Genesis 22:24,וּפִֽילַגְשֹׁ֖ו וּשְׁמָ֣הּ רְאוּמָ֑ה וַתֵּ֤לֶד גַּם־הִוא֙ אֶת־טֶ֣בַח וְאֶת־גַּ֔חַם וְאֶת־תַּ֖חַשׁ וְאֶֽת־מַעֲכָֽה׃ ס,וּפִֽילַגְשֹׁ֖ו וַתֵּ֤לֶד גַּם־הִוא֙ אֶת־טֶ֣בַח וְאֶת־גַּ֔חַם וְאֶת־תַּ֖חַשׁ וְאֶֽת־מַעֲכָֽה׃ ס,וּפִֽילַגְשֹׁ֖ו
6,Genesis 23:2,וַתָּ֣מָת שָׂרָ֗ה בְּקִרְיַ֥ת אַרְבַּ֛ע הִ֥וא חֶבְרֹ֖ון בְּאֶ֣רֶץ כְּנָ֑עַן וַיָּבֹא֙ אַבְרָהָ֔ם לִסְפֹּ֥ד לְשָׂרָ֖ה וְלִבְכֹּתָֽהּ׃,וַתָּ֣מָת שָׂרָ֗ה בְּקִרְיַ֥ת אַרְבַּ֛ע בְּאֶ֣רֶץ כְּנָ֑עַן,וַתָּ֣מָת שָׂרָ֗ה בְּקִרְיַ֥ת אַרְבַּ֛ע
7,Genesis 23:19,וְאַחֲרֵי־כֵן֩ קָבַ֨ר אַבְרָהָ֜ם אֶת־שָׂרָ֣ה אִשְׁתֹּ֗ו אֶל־מְעָרַ֞ת שְׂדֵ֧ה הַמַּכְפֵּלָ֛ה עַל־פְּנֵ֥י מַמְרֵ֖א הִ֣וא חֶבְרֹ֑ון בְּאֶ֖רֶץ כְּנָֽעַן׃,וְאַחֲרֵי־כֵן֩ קָבַ֨ר אַבְרָהָ֜ם אֶת־שָׂרָ֣ה אִשְׁתֹּ֗ו אֶל־מְעָרַ֞ת שְׂדֵ֧ה הַמַּכְפֵּלָ֛ה עַל־פְּנֵ֥י מַמְרֵ֖א בְּאֶ֖רֶץ כְּנָֽעַן׃,וְאַחֲרֵי־כֵן֩ קָבַ֨ר אַבְרָהָ֜ם אֶת־שָׂרָ֣ה אִשְׁתֹּ֗ו אֶל־מְעָרַ֞ת שְׂדֵ֧ה הַמַּכְפֵּלָ֛ה עַל־פְּנֵ֥י מַמְרֵ֖א
8,Genesis 24:7,יְהוָ֣ה׀ אֱלֹהֵ֣י הַשָּׁמַ֗יִם אֲשֶׁ֨ר לְקָחַ֜נִי מִבֵּ֣ית אָבִי֮ וּמֵאֶ֣רֶץ מֹֽולַדְתִּי֒ וַאֲשֶׁ֨ר דִּבֶּר־לִ֜י וַאֲשֶׁ֤ר נִֽשְׁבַּֽע־לִי֙ לֵאמֹ֔ר לְזַ֨רְעֲךָ֔ אֶתֵּ֖ן אֶת־הָאָ֣רֶץ הַזֹּ֑את ה֗וּא יִשְׁלַ֤ח מַלְאָכֹו֙ לְפָנֶ֔יךָ וְלָקַחְתָּ֥ אִשָּׁ֛ה לִבְנִ֖י מִשָּֽׁם׃,יְהוָ֣ה׀ אֱלֹהֵ֣י הַשָּׁמַ֗יִם אֲשֶׁ֨ר לְקָחַ֜נִי מִבֵּ֣ית אָבִי֮ וּמֵאֶ֣רֶץ מֹֽולַדְתִּי֒ וַאֲשֶׁ֨ר דִּבֶּר־לִ֜י וַאֲשֶׁ֤ר נִֽשְׁבַּֽע־לִי֙ לֵאמֹ֔ר ה֗וּא יִשְׁלַ֤ח מַלְאָכֹו֙ לְפָנֶ֔יךָ,יְהוָ֣ה׀ אֱלֹהֵ֣י הַשָּׁמַ֗יִם אֲשֶׁ֨ר לְקָחַ֜נִי מִבֵּ֣ית אָבִי֮ וּמֵאֶ֣רֶץ מֹֽולַדְתִּי֒ וַאֲשֶׁ֨ר דִּבֶּר־לִ֜י וַאֲשֶׁ֤ר נִֽשְׁבַּֽע־לִי֙ לֵאמֹ֔ר
9,Genesis 24:14,וְהָיָ֣ה הַֽנַּעֲרָ֗ אֲשֶׁ֨ר אֹמַ֤ר אֵלֶ֨יהָ֙ הַטִּי־נָ֤א כַדֵּךְ֙ וְאֶשְׁתֶּ֔ה וְאָמְרָ֣ה שְׁתֵ֔ה וְגַם־גְּמַלֶּ֖יךָ אַשְׁקֶ֑ה אֹתָ֤הּ הֹכַ֨חְתָּ֙ לְעַבְדְּךָ֣ לְיִצְחָ֔ק וּבָ֣הּ אֵדַ֔ע כִּי־עָשִׂ֥יתָ חֶ֖סֶד עִם־אֲדֹנִֽי׃,הַֽנַּעֲרָ֗ אֲשֶׁ֨ר אֹמַ֤ר אֵלֶ֨יהָ֙ וְאָמְרָ֣ה אֹתָ֤הּ הֹכַ֨חְתָּ֙ לְעַבְדְּךָ֣ לְיִצְחָ֔ק,הַֽנַּעֲרָ֗ אֲשֶׁ֨ר אֹמַ֤ר אֵלֶ֨יהָ֙
10,Genesis 24:49,וְ֠עַתָּה אִם־יֶשְׁכֶ֨ם עֹשִׂ֜ים חֶ֧סֶד וֶֽאֱמֶ֛ת אֶת־אֲדֹנִ֖י הַגִּ֣ידוּ לִ֑י וְאִם־לֹ֕א הַגִּ֣ידוּ לִ֔י וְאֶפְנֶ֥ה עַל־יָמִ֖ין אֹ֥ו עַל־שְׂמֹֽאל׃,אִם־יֶשְׁכֶ֨ם עֹשִׂ֜ים חֶ֧סֶד וֶֽאֱמֶ֛ת אֶת־אֲדֹנִ֖י וְאִם־לֹ֕א,אִם־יֶשְׁכֶ֨ם עֹשִׂ֜ים חֶ֧סֶד וֶֽאֱמֶ֛ת אֶת־אֲדֹנִ֖י


  0.56s Done


Now in the common domain.

In [24]:
indent(reset=True)
query = '''
verse
  s:common otype=sentence
    =: sa:sentence_atom
  s ## sa
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  0.95s 16 results


n,p,verse,sentence,sentence_atom
1,Numbers 23:19,לֹ֣א אִ֥ישׁ אֵל֙ וִֽיכַזֵּ֔ב וּבֶן־אָדָ֖ם וְיִתְנֶחָ֑ם הַה֤וּא אָמַר֙ וְלֹ֣א יַעֲשֶׂ֔ה וְדִבֶּ֖ר וְלֹ֥א יְקִימֶֽנָּה׃,הַה֤וּא אָמַר֙ וְדִבֶּ֖ר,הַה֤וּא אָמַר֙
2,Judges 7:4,וַיֹּ֨אמֶר יְהוָ֜ה אֶל־גִּדְעֹ֗ון עֹוד֮ הָעָ֣ם רָב֒ הֹורֵ֤ד אֹותָם֙ אֶל־הַמַּ֔יִם וְאֶצְרְפֶ֥נּוּ לְךָ֖ שָׁ֑ם וְהָיָ֡ה אֲשֶׁר֩ אֹמַ֨ר אֵלֶ֜יךָ זֶ֣ה׀ יֵלֵ֣ךְ אִתָּ֗ךְ ה֚וּא יֵלֵ֣ךְ אִתָּ֔ךְ וְכֹ֨ל אֲשֶׁר־אֹמַ֜ר אֵלֶ֗יךָ זֶ֚ה לֹא־יֵלֵ֣ךְ עִמָּ֔ךְ ה֖וּא לֹ֥א יֵלֵֽךְ׃,אֲשֶׁר֩ אֹמַ֨ר אֵלֶ֜יךָ ה֚וּא יֵלֵ֣ךְ אִתָּ֔ךְ,אֲשֶׁר֩ אֹמַ֨ר אֵלֶ֜יךָ
3,Judges 7:4,וַיֹּ֨אמֶר יְהוָ֜ה אֶל־גִּדְעֹ֗ון עֹוד֮ הָעָ֣ם רָב֒ הֹורֵ֤ד אֹותָם֙ אֶל־הַמַּ֔יִם וְאֶצְרְפֶ֥נּוּ לְךָ֖ שָׁ֑ם וְהָיָ֡ה אֲשֶׁר֩ אֹמַ֨ר אֵלֶ֜יךָ זֶ֣ה׀ יֵלֵ֣ךְ אִתָּ֗ךְ ה֚וּא יֵלֵ֣ךְ אִתָּ֔ךְ וְכֹ֨ל אֲשֶׁר־אֹמַ֜ר אֵלֶ֗יךָ זֶ֚ה לֹא־יֵלֵ֣ךְ עִמָּ֔ךְ ה֖וּא לֹ֥א יֵלֵֽךְ׃,וְכֹ֨ל אֲשֶׁר־אֹמַ֜ר אֵלֶ֗יךָ ה֖וּא לֹ֥א יֵלֵֽךְ׃,וְכֹ֨ל אֲשֶׁר־אֹמַ֜ר אֵלֶ֗יךָ
4,2_Samuel 5:6,וַיֵּ֨לֶךְ הַמֶּ֤לֶךְ וַֽאֲנָשָׁיו֙ יְר֣וּשָׁלִַ֔ם אֶל־הַיְבֻסִ֖י יֹושֵׁ֣ב הָאָ֑רֶץ וַיֹּ֨אמֶר לְדָוִ֤ד לֵאמֹר֙ לֹא־תָבֹ֣וא הֵ֔נָּה כִּ֣י אִם־הֱסִֽירְךָ֗ הַעִוְרִ֤ים וְהַפִּסְחִים֙ לֵאמֹ֔ר לֹֽא־יָבֹ֥וא דָוִ֖ד הֵֽנָּה׃,וַיֹּ֨אמֶר לְדָוִ֤ד לֵאמֹר֙ לֵאמֹ֔ר,וַיֹּ֨אמֶר לְדָוִ֤ד לֵאמֹר֙
5,2_Samuel 21:4,וַיֹּ֧אמְרוּ לֹ֣ו הַגִּבְעֹנִ֗ים אֵֽין־לָ֜נוּ כֶּ֤סֶף וְזָהָב֙ עִם־שָׁא֣וּל וְעִם־בֵּיתֹ֔ו וְאֵֽין־לָ֥נוּ אִ֖ישׁ לְהָמִ֣ית בְּיִשְׂרָאֵ֑ל וַיֹּ֛אמֶר מָֽה־אַתֶּ֥ם אֹמְרִ֖ים אֶעֱשֶׂ֥ה לָכֶֽם׃,מָֽה־אֶעֱשֶׂ֥ה לָכֶֽם׃,מָֽה־
6,Isaiah 30:16,וַתֹּ֨אמְר֥וּ לֹא־כִ֛י עַל־ס֥וּס נָנ֖וּס עַל־כֵּ֣ן תְּנוּס֑וּן וְעַל־קַ֣ל נִרְכָּ֔ב עַל־כֵּ֖ן יִקַּ֥לּוּ רֹדְפֵיכֶֽם׃,וַתֹּ֨אמְר֥וּ וְ,וַתֹּ֨אמְר֥וּ
7,Isaiah 41:17,הָעֲנִיִּ֨ים וְהָאֶבְיֹונִ֜ים מְבַקְשִׁ֥ים מַ֨יִם֙ וָאַ֔יִן לְשֹׁונָ֖ם בַּצָּמָ֣א נָשָׁ֑תָּה אֲנִ֤י יְהוָה֙ אֶעֱנֵ֔ם אֱלֹהֵ֥י יִשְׂרָאֵ֖ל לֹ֥א אֶעֶזְבֵֽם׃,אֲנִ֤י יְהוָה֙ אֱלֹהֵ֥י יִשְׂרָאֵ֖ל,אֲנִ֤י יְהוָה֙
8,Isaiah 41:28,וְאֵ֨רֶא֙ וְאֵ֣ין אִ֔ישׁ וּמֵאֵ֖לֶּה וְאֵ֣ין יֹועֵ֑ץ וְאֶשְׁאָלֵ֖ם וְיָשִׁ֥יבוּ דָבָֽר׃,וְאֵ֨רֶא֙ וּמֵאֵ֖לֶּה,וְאֵ֨רֶא֙
9,Isaiah 55:7,יַעֲזֹ֤ב רָשָׁע֙ דַּרְכֹּ֔ו וְאִ֥ישׁ אָ֖וֶן מַחְשְׁבֹתָ֑יו וְיָשֹׁ֤ב אֶל־יְהוָה֙ וִֽירַחֲמֵ֔הוּ וְאֶל־אֱלֹהֵ֖ינוּ כִּֽי־יַרְבֶּ֥ה לִסְלֹֽוחַ׃,וְיָשֹׁ֤ב אֶל־יְהוָה֙ וְאֶל־אֱלֹהֵ֖ינוּ,וְיָשֹׁ֤ב אֶל־יְהוָה֙
10,Isaiah 65:1,נִדְרַ֨שְׁתִּי֙ לְלֹ֣וא שָׁאָ֔לוּ נִמְצֵ֖אתִי לְלֹ֣א בִקְשֻׁ֑נִי אָמַ֨רְתִּי֙ הִנֵּ֣נִי הִנֵּ֔נִי אֶל־גֹּ֖וי לֹֽא־קֹרָ֥א בִשְׁמִֽי׃,אָמַ֨רְתִּי֙ אֶל־גֹּ֖וי לֹֽא־קֹרָ֥א בִשְׁמִֽי׃,אָמַ֨רְתִּי֙


  0.97s Done


## || disjoint slots

True when the two objects in question do not share any slots.
This is a rather loose relationship.

This can be used for locating gaps: a textual object that lies inside a gap of another object.
See also [gaps](searchGaps.ipynb).

Here we check whether there are phrases with disjoint subphrases.

In [25]:
indent(reset=True)
query = '''
verse
  p:phrase
    s1:subphrase
    < s2:subphrase
  s1 || s2
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, start=1, end=1)
info('Done')

  2.25s 167760 results


n,p,verse,phrase,subphrase,subphrase
1,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֵ֥ת הַשָּׁמַ֖יִם,אֵ֥ת הָאָֽרֶץ׃
2,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,תֹ֨הוּ֙ וָבֹ֔הוּ,תֹ֨הוּ֙,בֹ֔הוּ
3,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,עַל־פְּנֵ֣י תְהֹ֑ום,פְּנֵ֣י,תְהֹ֑ום
4,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,ר֣וּחַ אֱלֹהִ֔ים,ר֣וּחַ,אֱלֹהִ֔ים
5,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,עַל־פְּנֵ֥י הַמָּֽיִם׃,פְּנֵ֥י,הַמָּֽיִם׃
6,Genesis 1:4,וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,בֵּ֥ין הָאֹ֖ור,בֵ֥ין הַחֹֽשֶׁךְ׃
7,Genesis 1:5,וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום אֶחָֽד׃ פ,יֹ֥ום אֶחָֽד׃ פ,יֹ֥ום,אֶחָֽד׃ פ
8,Genesis 1:6,וַיֹּ֣אמֶר אֱלֹהִ֔ים יְהִ֥י רָקִ֖יעַ בְּתֹ֣וךְ הַמָּ֑יִם וִיהִ֣י מַבְדִּ֔יל בֵּ֥ין מַ֖יִם לָמָֽיִם׃,בְּתֹ֣וךְ הַמָּ֑יִם,תֹ֣וךְ,הַמָּ֑יִם
9,Genesis 1:8,וַיִּקְרָ֧א אֱלֹהִ֛ים לָֽרָקִ֖יעַ שָׁמָ֑יִם וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום שֵׁנִֽי׃ פ,יֹ֥ום שֵׁנִֽי׃ פ,יֹ֥ום,שֵׁנִֽי׃ פ
10,Genesis 1:9,וַיֹּ֣אמֶר אֱלֹהִ֗ים יִקָּו֨וּ הַמַּ֜יִם מִתַּ֤חַת הַשָּׁמַ֨יִם֙ אֶל־מָקֹ֣ום אֶחָ֔ד וְתֵרָאֶ֖ה הַיַּבָּשָׁ֑ה וַֽיְהִי־כֵֽן׃,הַמַּ֜יִם מִתַּ֤חַת הַשָּׁמַ֨יִם֙,תַּ֤חַת,הַשָּׁמַ֨יִם֙


  2.26s Done


Now in the common domain.

In [26]:
indent(reset=True)
query = '''
verse
  p:common otype=phrase
    s1:subphrase
    < s2:subphrase
  s1 || s2
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  1.29s 10958 results


n,p,verse,phrase,subphrase,subphrase
1,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,עַל־פְּנֵ֥י הַמָּֽיִם׃,פְּנֵ֥י,הַמָּֽיִם׃
2,Genesis 1:5,וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום אֶחָֽד׃ פ,יֹ֥ום אֶחָֽד׃ פ,יֹ֥ום,אֶחָֽד׃ פ
3,Genesis 1:29,וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃,עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ,פְּנֵ֣י,כָל־הָאָ֔רֶץ
4,Genesis 1:29,וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃,עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ,פְּנֵ֣י,כָל־
5,Genesis 1:29,וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃,עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ,פְּנֵ֣י,הָאָ֔רֶץ
6,Genesis 1:29,וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃,עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ,כָל־,הָאָ֔רֶץ
7,Genesis 2:4,אֵ֣לֶּה תֹולְדֹ֧ות הַשָּׁמַ֛יִם וְהָאָ֖רֶץ בְּהִבָּֽרְאָ֑ם בְּיֹ֗ום עֲשֹׂ֛ות יְהוָ֥ה אֱלֹהִ֖ים אֶ֥רֶץ וְשָׁמָֽיִם׃,יְהוָ֥ה אֱלֹהִ֖ים,יְהוָ֥ה,אֱלֹהִ֖ים
8,Genesis 2:5,וְכֹ֣ל׀ שִׂ֣יחַ הַשָּׂדֶ֗ה טֶ֚רֶם יִֽהְיֶ֣ה בָאָ֔רֶץ וְכָל־עֵ֥שֶׂב הַשָּׂדֶ֖ה טֶ֣רֶם יִצְמָ֑ח כִּי֩ לֹ֨א הִמְטִ֜יר יְהוָ֤ה אֱלֹהִים֙ עַל־הָאָ֔רֶץ וְאָדָ֣ם אַ֔יִן לַֽעֲבֹ֖ד אֶת־הָֽאֲדָמָֽה׃,יְהוָ֤ה אֱלֹהִים֙,יְהוָ֤ה,אֱלֹהִים֙
9,Genesis 2:7,וַיִּיצֶר֩ יְהוָ֨ה אֱלֹהִ֜ים אֶת־הָֽאָדָ֗ם עָפָר֙ מִן־הָ֣אֲדָמָ֔ה וַיִּפַּ֥ח בְּאַפָּ֖יו נִשְׁמַ֣ת חַיִּ֑ים וַֽיְהִ֥י הָֽאָדָ֖ם לְנֶ֥פֶשׁ חַיָּֽה׃,יְהוָ֨ה אֱלֹהִ֜ים,יְהוָ֨ה,אֱלֹהִ֜ים
10,Genesis 2:8,וַיִּטַּ֞ע יְהוָ֧ה אֱלֹהִ֛ים גַּן־בְּעֵ֖דֶן מִקֶּ֑דֶם וַיָּ֣שֶׂם שָׁ֔ם אֶת־הָֽאָדָ֖ם אֲשֶׁ֥ר יָצָֽר׃,יְהוָ֧ה אֱלֹהִ֛ים,יְהוָ֧ה,אֱלֹהִ֛ים


  1.31s Done


We see that Genesis 1:1 has fallen out. Let's check the ranks of the lexemes of its last phrase:

In [27]:
firstSentence = F.otype.s('sentence')[0]
lastPhrase = L.d(firstSentence, otype='phrase')[-1]
[(F.g_word_utf8.v(w), F.rank_lex.v(w)) for w in L.d(lastPhrase, otype='word')]

[('אֵ֥ת', 4),
 ('הַ', 1),
 ('שָּׁמַ֖יִם', 122),
 ('וְ', 0),
 ('אֵ֥ת', 4),
 ('הָ', 1),
 ('אָֽרֶץ', 22)]

There you have it: the heavens are rare!

## [[ and ]] embedding

`n [[ m` if object `n` embeds `m`.

`n ]] m` if object `n` lies embedded in `m`.

These relations are used implicitly in templates when there is indentation:

```
s:sentence
  p:phrase
    w1:word gn=f
    w2:word gn=m
```

The template above implicitly states the following embeddings:

* `s ]] p`
* `p ]] w1`
* `p ]] w2`

We have seen these relations in action.

# Positioning

## << and >> before and after (slot-wise)

These relations test whether one object comes before or after an other,
in the sense that the slots
occupied by the one object lie completely 
before or after the slots occupied by the other object.

In [28]:
query = '''
verse
  sentence
    c1:clause
    p:phrase
    c2:clause
    c1 << p
    c2 >> p
'''
results = A.search(query)
A.table(results, end=10)
A.show(results, condensed=False, start=1, end=1, colorMap={2: 'lightyellow', 3: 'cyan', 4: 'magenta', 5: 'blue'})

  2.66s 30933 results


n,p,verse,sentence,clause,phrase,clause
1,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י,עֹ֤שֶׂה,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
2,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י,פְּרִי֙,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
3,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י,לְמִינֹ֔ו,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
4,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ,מַזְרִ֣יעַ זֶ֔רַע,עֹ֤שֶׂה,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
5,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ,מַזְרִ֣יעַ זֶ֔רַע,פְּרִי֙,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
6,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ,מַזְרִ֣יעַ זֶ֔רַע,לְמִינֹ֔ו,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
7,Genesis 1:12,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ,עֹ֥שֶׂה,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
8,Genesis 1:12,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ,פְּרִ֛י,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
9,Genesis 1:12,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ,מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ,עֹ֥שֶׂה,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו
10,Genesis 1:12,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ,מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ,פְּרִ֛י,אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו


In the common domain:

In [29]:
indent(reset=True)
query = '''
verse
  sentence
    c1:common otype=clause
    p:rare otype=phrase
    c2:common otype=clause
    c1 << p
    c2 >> p
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  2.33s 236 results


n,p,verse,sentence,clause,phrase,clause
1,Genesis 3:5,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים,נִפְקְח֖וּ,וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים
2,Genesis 3:5,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,כִּ֗י בְּיֹום֙,נִפְקְח֖וּ,וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים
3,Genesis 3:5,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,כִּ֚י יֹדֵ֣עַ אֱלֹהִ֔ים כִּ֗י בְּיֹום֙ אֲכָלְכֶ֣ם מִמֶּ֔נּוּ וְנִפְקְח֖וּ עֵֽינֵיכֶ֑ם וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים יֹדְעֵ֖י טֹ֥וב וָרָֽע׃,אֲכָלְכֶ֣ם מִמֶּ֔נּוּ,נִפְקְח֖וּ,וִהְיִיתֶם֙ כֵּֽאלֹהִ֔ים
4,Genesis 3:12,וַיֹּ֖אמֶר הָֽאָדָ֑ם הָֽאִשָּׁה֙ אֲשֶׁ֣ר נָתַ֣תָּה עִמָּדִ֔י הִ֛וא נָֽתְנָה־לִּ֥י מִן־הָעֵ֖ץ וָאֹכֵֽל׃,הָֽאִשָּׁה֙ אֲשֶׁ֣ר נָתַ֣תָּה עִמָּדִ֔י הִ֛וא נָֽתְנָה־לִּ֥י מִן־הָעֵ֖ץ וָאֹכֵֽל׃,הָֽאִשָּׁה֙,עִמָּדִ֔י,וָאֹכֵֽל׃
5,Genesis 14:17,וַיֵּצֵ֣א מֶֽלֶךְ־סְדֹם֮ לִקְרָאתֹו֒ אַחֲרֵ֣י שׁוּבֹ֗ו מֵֽהַכֹּות֙ אֶת־כְּדָר־לָעֹ֔מֶר וְאֶת־הַמְּלָכִ֖ים אֲשֶׁ֣ר אִתֹּ֑ו אֶל־עֵ֣מֶק שָׁוֵ֔ה ה֖וּא עֵ֥מֶק הַמֶּֽלֶךְ׃,וַיֵּצֵ֣א מֶֽלֶךְ־סְדֹם֮ לִקְרָאתֹו֒ אַחֲרֵ֣י שׁוּבֹ֗ו מֵֽהַכֹּות֙ אֶת־כְּדָר־לָעֹ֔מֶר וְאֶת־הַמְּלָכִ֖ים אֲשֶׁ֣ר אִתֹּ֑ו אֶל־עֵ֣מֶק שָׁוֵ֔ה,אַחֲרֵ֣י שׁוּבֹ֗ו,אֶת־כְּדָר־לָעֹ֔מֶר וְאֶת־הַמְּלָכִ֖ים,אֲשֶׁ֣ר אִתֹּ֑ו
6,Genesis 15:7,וַיֹּ֖אמֶר אֵלָ֑יו אֲנִ֣י יְהוָ֗ה אֲשֶׁ֤ר הֹוצֵאתִ֨יךָ֙ מֵא֣וּר כַּשְׂדִּ֔ים לָ֧תֶת לְךָ֛ אֶת־הָאָ֥רֶץ הַזֹּ֖את לְרִשְׁתָּֽהּ׃,אֲנִ֣י יְהוָ֗ה אֲשֶׁ֤ר הֹוצֵאתִ֨יךָ֙ מֵא֣וּר כַּשְׂדִּ֔ים לָ֧תֶת לְךָ֛ אֶת־הָאָ֥רֶץ הַזֹּ֖את לְרִשְׁתָּֽהּ׃,אֲנִ֣י יְהוָ֗ה,מֵא֣וּר כַּשְׂדִּ֔ים,לָ֧תֶת לְךָ֛ אֶת־הָאָ֥רֶץ הַזֹּ֖את
7,Genesis 18:21,אֵֽרֲדָה־נָּ֣א וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א אֵדָֽעָה׃,וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א,וְאֶרְאֶ֔ה,כְּצַעֲקָתָ֛הּ,הַבָּ֥אָה אֵלַ֖י
8,Genesis 18:21,אֵֽרֲדָה־נָּ֣א וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א אֵדָֽעָה׃,וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א,וְאֶרְאֶ֔ה,כְּצַעֲקָתָ֛הּ,וְאִם־לֹ֖א
9,Genesis 18:21,אֵֽרֲדָה־נָּ֣א וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א אֵדָֽעָה׃,וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א,וְאֶרְאֶ֔ה,כָּלָ֑ה,וְאִם־לֹ֖א
10,Genesis 18:21,אֵֽרֲדָה־נָּ֣א וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א אֵדָֽעָה׃,וְאֶרְאֶ֔ה הַכְּצַעֲקָתָ֛הּ הַבָּ֥אָה אֵלַ֖י עָשׂ֣וּ׀ כָּלָ֑ה וְאִם־לֹ֖א,הַבָּ֥אָה אֵלַ֖י,כָּלָ֑ה,וְאִם־לֹ֖א


  2.35s Done


## =: same start slots
This relation holds when the left and right hand sides are nodes that have the same first slot.
It serves to enforce the the children of a parent are textually the first things inside that
parent. We have seen it in action before.

## := same end slots
This relation holds when the left and right hand sides are nodes that have the same last slot
It serves to enforce the the children of a parent are textually the last things inside that
parent. We have seen it in action before.

## :: same boundary slots
This relation holds when `=:` and `:=` both hold between the left and right hand sides.
It serves to look for parents with single children, or at least, where the parent is textually spanned by a single child.

Let us look for a phrase, whose start and end slots coincide with its containing clause.
But only if the phrase does not conincide with its parent clause.

In [30]:
query = '''
verse
  c:clause
    :: p:phrase
c ## p
'''
results = A.search(query)
A.table(results, start=1, end=10)
A.show(results, start=1, end=5, condenseType='clause')

  0.47s 54 results


n,p,verse,clause,phrase
1,Genesis 10:21,וּלְשֵׁ֥ם יֻלַּ֖ד גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃,גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃,גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַגָּדֹֽול׃
2,Genesis 24:24,וַתֹּ֣אמֶר אֵלָ֔יו בַּת־בְּתוּאֵ֖ל אָנֹ֑כִי בֶּן־מִלְכָּ֕ה אֲשֶׁ֥ר יָלְדָ֖ה לְנָחֹֽור׃,בַּת־בְּתוּאֵ֖ל אָנֹ֑כִי בֶּן־מִלְכָּ֕ה,בַּת־בְּתוּאֵ֖ל בֶּן־מִלְכָּ֕ה
3,Genesis 31:16,כִּ֣י כָל־הָעֹ֗שֶׁר אֲשֶׁ֨ר הִצִּ֤יל אֱלֹהִים֙ מֵֽאָבִ֔ינוּ לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ וְעַתָּ֗ה כֹּל֩ אֲשֶׁ֨ר אָמַ֧ר אֱלֹהִ֛ים אֵלֶ֖יךָ עֲשֵֽׂה׃,לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ,לָ֥נוּ וּלְבָנֵ֑ינוּ
4,Genesis 31:53,אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ יִשְׁפְּט֣וּ בֵינֵ֔ינוּ אֱלֹהֵ֖י אֲבִיהֶ֑ם וַיִּשָּׁבַ֣ע יַעֲקֹ֔ב בְּפַ֖חַד אָבִ֥יו יִצְחָֽק׃,אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ יִשְׁפְּט֣וּ בֵינֵ֔ינוּ אֱלֹהֵ֖י אֲבִיהֶ֑ם,אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ אֱלֹהֵ֖י אֲבִיהֶ֑ם
5,Exodus 28:1,וְאַתָּ֡ה הַקְרֵ֣ב אֵלֶיךָ֩ אֶת־אַהֲרֹ֨ן אָחִ֜יךָ וְאֶת־בָּנָ֣יו אִתֹּ֔ו מִתֹּ֛וךְ בְּנֵ֥י יִשְׂרָאֵ֖ל לְכַהֲנֹו־לִ֑י אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃,לְכַהֲנֹו־לִ֑י אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃,לְכַהֲנֹו־אַהֲרֹ֕ן נָדָ֧ב וַאֲבִיה֛וּא אֶלְעָזָ֥ר וְאִיתָמָ֖ר בְּנֵ֥י אַהֲרֹֽן׃
6,Exodus 28:14,וּשְׁתֵּ֤י שַׁרְשְׁרֹת֙ זָהָ֣ב טָהֹ֔ור מִגְבָּלֹ֛ת תַּעֲשֶׂ֥ה אֹתָ֖ם מַעֲשֵׂ֣ה עֲבֹ֑ת וְנָתַתָּ֛ה אֶת־שַׁרְשְׁרֹ֥ת הָעֲבֹתֹ֖ת עַל־הַֽמִּשְׁבְּצֹֽת׃ ס,מִגְבָּלֹ֛ת תַּעֲשֶׂ֥ה אֹתָ֖ם מַעֲשֵׂ֣ה עֲבֹ֑ת,מִגְבָּלֹ֛ת מַעֲשֵׂ֣ה עֲבֹ֑ת
7,Exodus 29:18,וְהִקְטַרְתָּ֤ אֶת־כָּל־הָאַ֨יִל֙ הַמִּזְבֵּ֔חָה עֹלָ֥ה ה֖וּא לַֽיהוָ֑ה רֵ֣יחַ נִיחֹ֔וחַ אִשֶּׁ֥ה לַיהוָ֖ה הֽוּא׃,עֹלָ֥ה ה֖וּא לַֽיהוָ֑ה רֵ֣יחַ נִיחֹ֔וחַ,עֹלָ֥ה רֵ֣יחַ נִיחֹ֔וחַ
8,Exodus 30:4,וּשְׁתֵּי֩ טַבְּעֹ֨ת זָהָ֜ב תַּֽעֲשֶׂה־לֹּ֣ו׀ מִתַּ֣חַת לְזֵרֹ֗ו עַ֚ל שְׁתֵּ֣י צַלְעֹתָ֔יו תַּעֲשֶׂ֖ה עַל־שְׁנֵ֣י צִדָּ֑יו וְהָיָה֙ לְבָתִּ֣ים לְבַדִּ֔ים לָשֵׂ֥את אֹתֹ֖ו בָּהֵֽמָּה׃,עַ֚ל שְׁתֵּ֣י צַלְעֹתָ֔יו תַּעֲשֶׂ֖ה עַל־שְׁנֵ֣י צִדָּ֑יו,עַ֚ל שְׁתֵּ֣י צַלְעֹתָ֔יו עַל־שְׁנֵ֣י צִדָּ֑יו
9,Exodus 30:13,זֶ֣ה׀ יִתְּנ֗וּ כָּל־הָעֹבֵר֙ עַל־הַפְּקֻדִ֔ים מַחֲצִ֥ית הַשֶּׁ֖קֶל בְּשֶׁ֣קֶל הַקֹּ֑דֶשׁ עֶשְׂרִ֤ים גֵּרָה֙ הַשֶּׁ֔קֶל מַחֲצִ֣ית הַשֶּׁ֔קֶל תְּרוּמָ֖ה לַֽיהוָֽה׃,זֶ֣ה׀ יִתְּנ֗וּ כָּל־מַחֲצִ֥ית הַשֶּׁ֖קֶל בְּשֶׁ֣קֶל הַקֹּ֑דֶשׁ,זֶ֣ה׀ מַחֲצִ֥ית הַשֶּׁ֖קֶל בְּשֶׁ֣קֶל הַקֹּ֑דֶשׁ
10,Exodus 36:35,וַיַּ֨עַשׂ֙ אֶת־הַפָּרֹ֔כֶת תְּכֵ֧לֶת וְאַרְגָּמָ֛ן וְתֹולַ֥עַת שָׁנִ֖י וְשֵׁ֣שׁ מָשְׁזָ֑ר מַעֲשֵׂ֥ה חֹשֵׁ֛ב עָשָׂ֥ה אֹתָ֖הּ כְּרֻבִֽים׃,מַעֲשֵׂ֥ה חֹשֵׁ֛ב עָשָׂ֥ה אֹתָ֖הּ כְּרֻבִֽים׃,מַעֲשֵׂ֥ה חֹשֵׁ֛ב כְּרֻבִֽים׃


Here you see an extra phrase in such clauses, lying embedded in the clause-spanning phrase.

A nice case of **Mind the gap!**.

In the common domain:

In [31]:
indent(reset=True)
query = '''
verse
  c:common otype=clause
    :: p:common otype=phrase
c ## p
'''
resultsCustom = A.search(query, sets=customSets)
A.table(resultsCustom, end=10)
A.show(resultsCustom, start=1, end=1)
info('Done')

  1.67s 3 results


n,p,verse,clause,phrase
1,Genesis 31:16,כִּ֣י כָל־הָעֹ֗שֶׁר אֲשֶׁ֨ר הִצִּ֤יל אֱלֹהִים֙ מֵֽאָבִ֔ינוּ לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ וְעַתָּ֗ה כֹּל֩ אֲשֶׁ֨ר אָמַ֧ר אֱלֹהִ֛ים אֵלֶ֖יךָ עֲשֵֽׂה׃,לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ,לָ֥נוּ וּלְבָנֵ֑ינוּ
2,Leviticus 11:9,אֶת־זֶה֙ תֹּֽאכְל֔וּ מִכֹּ֖ל אֲשֶׁ֣ר בַּמָּ֑יִם כֹּ֣ל אֲשֶׁר־לֹו֩ סְנַפִּ֨יר וְקַשְׂקֶ֜שֶׂת בַּמַּ֗יִם בַּיַּמִּ֛ים וּבַנְּחָלִ֖ים אֹתָ֥ם תֹּאכֵֽלוּ׃,אֶת־זֶה֙ תֹּֽאכְל֔וּ מִכֹּ֖ל,אֶת־זֶה֙ מִכֹּ֖ל
3,Ezekiel 14:4,לָכֵ֣ן דַּבֵּר־אֹ֠ותָם וְאָמַרְתָּ֨ אֲלֵיהֶ֜ם כֹּה־אָמַ֣ר׀ אֲדֹנָ֣י יְהוִ֗ה אִ֣ישׁ אִ֣ישׁ מִבֵּ֣ית יִשְׂרָאֵ֡ל אֲשֶׁר֩ יַעֲלֶ֨ה אֶת־גִּלּוּלָ֜יו אֶל־לִבֹּ֗ו וּמִכְשֹׁ֤ול עֲוֹנֹו֙ יָשִׂים֙ נֹ֣כַח פָּנָ֔יו וּבָ֖א אֶל־הַנָּבִ֑יא אֲנִ֣י יְהוָ֗ה נַעֲנֵ֧יתִי לֹ֦ו בָ֖א בְּרֹ֥ב גִּלּוּלָֽיו׃,אִ֣ישׁ אִ֣ישׁ מִבֵּ֣ית יִשְׂרָאֵ֡ל,אִ֣ישׁ מִבֵּ֣ית יִשְׂרָאֵ֡ל


  1.68s Done


## <: adjacent before 
This relation holds when the left hand sides ends in a slot that lies before the first slot of the right hand side.
It serves to enforce an ordering between siblings of a parent.

## :> adjacent after
This relation holds when the left hand sides starts in a slot that lies after the last slot of the right hand side.

In [32]:
query = '''
verse
  clause
    phrase
    <: phrase
'''
results = A.search(query)
A.table(results, start=1, end=3)

  2.41s 162555 results


n,p,verse,clause,phrase,phrase
1,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית,בָּרָ֣א
2,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בָּרָ֣א,אֱלֹהִ֑ים
3,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֱלֹהִ֑ים,אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃


In [33]:
query = '''
verse
  clause
    phrase
    :> phrase
'''
results = A.search(query)
A.table(results, start=1, end=3)

  2.40s 162555 results


n,p,verse,clause,phrase,phrase
1,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בָּרָ֣א,בְּרֵאשִׁ֖ית
2,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֱלֹהִ֑ים,בָּרָ֣א
3,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֱלֹהִ֑ים


Playing with common and rare:

In [34]:
query = '''
verse
  clause
    common otype=phrase
    <: rare otype=phrase
'''
resultsCommon = A.search(query, sets=customSets)
A.table(resultsCommon, start=1, end=3)

  2.01s 25894 results


n,p,verse,clause,phrase,phrase
1,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,הָיְתָ֥ה,תֹ֨הוּ֙ וָבֹ֔הוּ
2,Genesis 1:4,וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,כִּי־טֹ֑וב,כִּי־,טֹ֑וב
3,Genesis 1:4,וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,וַ,יַּבְדֵּ֣ל


In [35]:
query = '''
verse
  clause
    common otype=phrase
    :> rare otype=phrase
'''
resultsCommon = A.search(query, sets=customSets)
A.table(resultsCommon, start=1, end=3)

  1.93s 9671 results


n,p,verse,clause,phrase,phrase
1,Genesis 1:1,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,אֱלֹהִ֑ים,בָּרָ֣א
2,Genesis 1:2,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,עַל־פְּנֵ֥י הַמָּֽיִם׃,מְרַחֶ֖פֶת
3,Genesis 1:4,וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃,אֱלֹהִ֔ים,יַּבְדֵּ֣ל


Another example: are there clauses with multiple clause atoms without a gap between the two?

In [36]:
query = '''
verse
  clause
    clause_atom
    <: clause_atom
'''
results = A.search(query)
A.table(results, start=1, end=10)

  0.75s 0 results


Conclusion: there is always textual material between clause_atoms of the same clause.
If we lift the adjacency to sequentially before (`<<`) we do get results:

In [37]:
query = '''
verse
  clause
    clause_atom
    << clause_atom
'''
results = A.search(query)
A.table(results, start=1, end=10)
A.show(results, start=1, end=1)

  0.70s 2589 results


n,p,verse,clause,clause_atom,clause_atom
1,Genesis 1:7,וַיַּ֣עַשׂ אֱלֹהִים֮ אֶת־הָרָקִיעַ֒ וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ וּבֵ֣ין הַמַּ֔יִם אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ וַֽיְהִי־כֵֽן׃,וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ וּבֵ֣ין הַמַּ֔יִם,וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙,וּבֵ֣ין הַמַּ֔יִם
2,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב עֵ֣ץ פְּרִ֞י,תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב,עֵ֣ץ פְּרִ֞י
3,Genesis 1:11,וַיֹּ֣אמֶר אֱלֹהִ֗ים תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב מַזְרִ֣יעַ זֶ֔רַע עֵ֣ץ פְּרִ֞י עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו עַל־הָאָ֑רֶץ וַֽיְהִי־כֵֽן׃,עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו עַל־הָאָ֑רֶץ,עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו,עַל־הָאָ֑רֶץ
4,Genesis 1:12,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב וְעֵ֧ץ,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב,וְעֵ֧ץ
5,Genesis 1:12,וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא עֵ֣שֶׂב מַזְרִ֤יעַ זֶ֨רַע֙ לְמִינֵ֔הוּ וְעֵ֧ץ עֹ֥שֶׂה פְּרִ֛י אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו לְמִינֵ֑הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,עֹ֥שֶׂה פְּרִ֛י לְמִינֵ֑הוּ,עֹ֥שֶׂה פְּרִ֛י,לְמִינֵ֑הוּ
6,Genesis 1:21,וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ הַֽחַיָּ֣ה׀ הָֽרֹמֶ֡שֶׂת אֲשֶׁר֩ שָׁרְצ֨וּ הַמַּ֜יִם לְמִֽינֵהֶ֗ם וְאֵ֨ת כָּל־עֹ֤וף כָּנָף֙ לְמִינֵ֔הוּ וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃,וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ לְמִֽינֵהֶ֗ם,וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַתַּנִּינִ֖ם הַגְּדֹלִ֑ים וְאֵ֣ת כָּל־נֶ֣פֶשׁ,לְמִֽינֵהֶ֗ם
7,Genesis 1:29,וַיֹּ֣אמֶר אֱלֹהִ֗ים הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ זֹרֵ֣עַ זֶ֗רַע אֲשֶׁר֙ עַל־פְּנֵ֣י כָל־הָאָ֔רֶץ וְאֶת־כָּל־הָעֵ֛ץ אֲשֶׁר־בֹּ֥ו פְרִי־עֵ֖ץ זֹרֵ֣עַ זָ֑רַע לָכֶ֥ם יִֽהְיֶ֖ה לְאָכְלָֽה׃,הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀ וְאֶת־כָּל־הָעֵ֛ץ,הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־עֵ֣שֶׂב׀,וְאֶת־כָּל־הָעֵ֛ץ
8,Genesis 1:30,וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֲשֶׁר־בֹּו֙ נֶ֣פֶשׁ חַיָּ֔ה אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה וַֽיְהִי־כֵֽן׃,וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה,וּֽלְכָל־חַיַּ֣ת הָ֠אָרֶץ וּלְכָל־עֹ֨וף הַשָּׁמַ֜יִם וּלְכֹ֣ל׀ רֹומֵ֣שׂ עַל־הָאָ֗רֶץ,אֶת־כָּל־יֶ֥רֶק עֵ֖שֶׂב לְאָכְלָ֑ה
9,Genesis 2:17,וּמֵעֵ֗ץ הַדַּ֨עַת֙ טֹ֣וב וָרָ֔ע לֹ֥א תֹאכַ֖ל מִמֶּ֑נּוּ כִּ֗י בְּיֹ֛ום אֲכָלְךָ֥ מִמֶּ֖נּוּ מֹ֥ות תָּמֽוּת׃,כִּ֗י בְּיֹ֛ום מֹ֥ות תָּמֽוּת׃,כִּ֗י בְּיֹ֛ום,מֹ֥ות תָּמֽוּת׃
10,Genesis 2:22,וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע אֲשֶׁר־לָקַ֥ח מִן־הָֽאָדָ֖ם לְאִשָּׁ֑ה וַיְבִאֶ֖הָ אֶל־הָֽאָדָֽם׃,וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע לְאִשָּׁ֑ה,וַיִּבֶן֩ יְהוָ֨ה אֱלֹהִ֧ים׀ אֶֽת־הַצֵּלָ֛ע,לְאִשָּׁ֑ה


# Nearness

The relations with `:` in their name always have a requirement somewhere that a slot of the
left hand node equals a slot of the right hand node, or that the two are adjacent.

All these relationships can be relaxed by a **nearness number**.
If you put a number *k* inside the relationship symbols, those restrictions will be relaxed to
*the one slot and the other slot should have a mutual distance of at most k*.

## =k: same start within k slots

Here is an example.

First we look for clauses, with a phrase in it that starts at the
same slot as the clause.

In [38]:
results = A.search('''
chapter book=Genesis chapter=1
  clause
    =: phrase
''')

  0.40s 126 results


Now we add a bit of freedom, but not much: 0. Indeed, this is no extra
freedom, and it should give the same number of results.

In [39]:
results=A.search('''
chapter book=Genesis chapter=1
  clause
    =0: phrase
''')

  0.39s 126 results


Now we add real freedom: 1 and 2

In [40]:
results = A.search('''
chapter book=Genesis chapter=1
    clause
      =1: phrase
''')

  0.40s 236 results


In [41]:
results = A.search('''
chapter book=Genesis chapter=1
    clause
      =2: phrase
''')

  0.41s 315 results


Let us see some cases:

In [42]:
A.table(results, start=1, end=10, linked=2)
A.show(results, condensed=False, start=1, end=4, colorMap={2: 'yellow', 3: 'cyan'})

n,p,chapter,clause,phrase
1,Genesis 1:1,,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית
2,Genesis 1:1,,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בָּרָ֣א
3,Genesis 1:2,,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,וְ
4,Genesis 1:2,,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,הָאָ֗רֶץ
5,Genesis 1:2,,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,וְ
6,Genesis 1:2,,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,חֹ֖שֶׁךְ
7,Genesis 1:2,,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,עַל־פְּנֵ֣י תְהֹ֑ום
8,Genesis 1:2,,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,וְ
9,Genesis 1:2,,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,ר֣וּחַ אֱלֹהִ֔ים
10,Genesis 1:3,,וַיֹּ֥אמֶר אֱלֹהִ֖ים,וַ


The first and second result show the same clause, with its first and second phrase respectively.

Note that we look for phrases that lie embedded in their clause.
So we do not get phrases of a preceding clause.

But if we want, we can get those as well.

In [43]:
results = A.search('''
chapter book=Genesis chapter=1
  c:clause
  p:phrase
    
  c =2: p
''')

  0.34s 485 results


We have more results now. Here is a closer look:

In [44]:
A.table(results, start=1, end=10)
A.show(results, condensed=False, start=12, end=14, colorMap={2: 'yellow', 3: 'cyan'})

n,p,chapter,clause,phrase
1,Genesis 1:1,,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בְּרֵאשִׁ֖ית
2,Genesis 1:1,,בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃,בָּרָ֣א
3,Genesis 1:2,,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,וְ
4,Genesis 1:2,,וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ וָבֹ֔הוּ,הָאָ֗רֶץ
5,Genesis 1:2,,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,וְ
6,Genesis 1:2,,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,חֹ֖שֶׁךְ
7,Genesis 1:2,,וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום,עַל־פְּנֵ֣י תְהֹ֑ום
8,Genesis 1:2,,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,וְ
9,Genesis 1:2,,וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־פְּנֵ֥י הַמָּֽיִם׃,ר֣וּחַ אֱלֹהִ֔ים
10,Genesis 1:3,,וַיֹּ֥אמֶר אֱלֹהִ֖ים,וַ


Here you see in results 13 and 14 a phrase of the previous clause.

Lets also play with common and rare:

In [45]:
resultsCommon = A.search('''
verse
  clause
    =: rare otype=phrase
''', sets=customSets)

  0.83s 10294 results


In [46]:
resultsCommon = A.search('''
verse
  clause
    =0: rare otype=phrase
''', sets=customSets)

  1.12s 10294 results


In [47]:
resultsCommon = A.search('''
verse
  clause
    =1: rare otype=phrase
''', sets=customSets)

  1.76s 26074 results


In [48]:
resultsCommon = A.search('''
verse
  clause
    =2: rare otype=phrase
''', sets=customSets)

  2.54s 39312 results


## :k= same end within k slots

## :k: same start and end within k slots

# Next

You are comfortable in space now.

Ready to enter a whole new dimension?
[quantifiers](searchQuantifiers.ipynb)

---

[basic](search.ipynb)
[advanced](searchAdvanced.ipynb)
[sets](searchSets.ipynb)
relations
[quantifiers](searchQuantifiers.ipynb)
[rough](searchRough.ipynb)
[gaps](searchGaps.ipynb)